# Test the added function of the class DistanceMetrics in validation.py

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
from pathlib import Path
import numpy as np

ROOT_DIR = Path(os.path.dirname(os.path.abspath(".gitignore"))).parent
os.chdir(ROOT_DIR)

In [3]:
# Load neccessary modules
import validation
import pandas as pd

In [4]:
df = pd.read_csv(str(ROOT_DIR) + '/results/sweden-20200903_172506/distance-metrics-national.csv')
df = df.loc[:, ['distance', 'sampers_sum', 'model_sum']]
df.head()

,distance,sampers_sum,model_sum
0,"(-0.001, 22.405]",0.0,0.0
1,"(22.405, 35.395]",0.0,0.0
2,"(35.395, 46.554]",0.0,0.0
3,"(46.554, 56.061]",0.0,0.0
4,"(56.061, 64.93]",0.0,0.0


In [21]:
titles = ['sampers', 'model']
dis = df.loc[(df[titles[0] + '_sum'] != 0) | (df[titles[1] + '_sum'] != 0), :]

In [5]:
test_1 = np.array([0.1, 0.1, 0.3, 0, 0.05, 0.02, 0.03, 0, 0.1, 0.1, 0.2, 0])
test_2 = np.array([0.1, 0.3, 0.1, 0, 0.1, 0.03, 0, 0.01, 0.09, 0.07, 0.1, 0.1])
print(sum(test_1), sum(test_2))
L = 12
zeros = list(np.where(test_1 == 0)[0]) + list(np.where(test_2 == 0)[0])
zeros = list(set(zeros))

tomerge = list(set(zeros + [x + 1 for x in zeros if x < L - 1] + [x - 1 for x in zeros if x == L - 1]))
print(tomerge)

1.0 1.0
[3, 4, 6, 7, 8, 10, 11]


In [27]:
test_1 = dis[titles[0] + '_sum'].values
test_2 = dis[titles[1] + '_sum'].values
print(sum(test_1), sum(test_2))
L = len(test_1)
zeros = list(np.where(test_1 == 0)[0]) + list(np.where(test_2 == 0)[0])
zeros = list(set(zeros))

tomerge = list(set(zeros + [x + 1 for x in zeros if x < L - 1] + [x - 1 for x in zeros if x == L - 1]))
tomerge = sorted(tomerge)
print(tomerge)

1.0000000000000002 1.0
[78, 79, 89, 90]


In [28]:
if not tomerge:
    out_1 = test_1
    out_2 = test_2
else:
    out_1 = test_1[0:tomerge[0]]
    out_2 = test_2[0:tomerge[0]]
    end_ind = tomerge[0]
    start_ind = end_ind
    i = 0
    while i < len(tomerge):
        if end_ind < tomerge[i]:
            end_ind = tomerge[i]
            out_1 = np.append(out_1, test_1[start_ind:end_ind])
            out_2 = np.append(out_2, test_2[start_ind:end_ind])
            start_ind = end_ind
        else:
            if (i == len(tomerge) - 1) | (tomerge[min(len(tomerge) - 1, i + 1)] - tomerge[i] > 1):
                end_ind = tomerge[i] + 1
                out_1 = np.append(out_1, [sum(test_1[start_ind:end_ind])])
                out_2 = np.append(out_2, [sum(test_2[start_ind:end_ind])])
                start_ind = end_ind
            else:
                end_ind = tomerge[i + 1]
            i = i + 1

In [29]:
sum(out_1 * np.log10(out_1 / out_2))

0.0236850380860222

In [30]:
# Sampers vs the proposed model - test the function
ds = validation.DistanceMetrics()
print(ds.kullback_leibler(distances=df, titles =['sampers', 'model']))

0.0236850380860222
